In [1]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
17,application_1684389228849_0027,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%configure -f
{
  "conf": {
    "spark.sql.extensions":  "io.delta.sql.DeltaSparkSessionExtension",
     "spark.sql.catalog.spark_catalog":  "org.apache.spark.sql.delta.catalog.DeltaCatalog",
      "spark.jars":"hdfs:///apps/delta/lib/delta-core_2.12-2.0.0.jar",
      "spark.jars.packages": "io.delta:delta-core_2.12:2.0.0"
  }
}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
18,application_1684389228849_0028,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
18,application_1684389228849_0028,pyspark,idle,Link,Link,None,✔


In [3]:
from pyspark.sql import SparkSession

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
# import pyspark


# builder = pyspark.sql.SparkSession.builder.appName("Delta_App") \
#     .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
#     .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

# spark = configure_spark_with_delta_pip(builder).getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# spark = SparkSession.builder \
#     .appName("DeltaLake") \
#     .config("spark.jars.packages", "io.delta:delta-core_2.12:2.2.0") \
#     .getOrCreate()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
# %pip install delta-spark==1.1.0

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
# %pip show delta-spark

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
from delta.tables import *
from pyspark.sql.functions import *

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Generate Dummy Dataset for testing

from datetime import datetime
dest = ["Seattle", "New York", "New Jersey", "Los Angeles", "Las Vegas", "Tucson","Washington DC","Philadelphia","Miami","San Francisco"]
def get_json_data(start, count, dest):
    time_stamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
    data = [{"trip_id": i, "ts": time_stamp, "route_id": chr(65 + (i % 10)), "destination": dest[i%10]} for i in range(start, start + count)]
    return data

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
#Create a spark dataframe
def create_json_df(spark, data):
    sc = spark.sparkContext
    return spark.read.json(sc.parallelize(data))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
config = {
    "table_name": "delta_trips_table",
    "target": "s3://emrpocgigaforce/Output/delta_lake_data/delta_table/",
    "primary_key": "trip_id"
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#Generating dataset of large size
import time
start_time=time.time()
df = create_json_df(spark, get_json_data(0, 2000000, dest))
end_time=time.time()
load_duration=end_time-start_time
print("generate_duration: {:.2f} sec".format(load_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

generate_duration: 30.23 sec

In [14]:
# Checking the memory 
from pyspark.serializers import PickleSerializer, AutoBatchedSerializer
def _to_java_object_rdd(rdd):  
    """ Return a JavaRDD of Object by unpickling
    It will convert each Python object into Java object by Pyrolite, whenever the
    RDD is serialized in batch or not.
    """
    rdd = rdd._reserialize(AutoBatchedSerializer(PickleSerializer()))
    return rdd.ctx._jvm.org.apache.spark.mllib.api.python.SerDe.pythonToJava(rdd._jrdd, True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
JavaObj = _to_java_object_rdd(df.rdd)
nbytes = sc._jvm.org.apache.spark.util.SizeEstimator.estimate(JavaObj)
print("Memory usage of DataFrame: {:.2f} MB".format(nbytes/1000000))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Memory usage of DataFrame: 87.22 MB

In [18]:
# Write the data in S3 
start_time=time.time()
df.write.mode("overwrite").format("delta").save(config['target'])
end_time=time.time()
write_duration=end_time-start_time
print("write_duration: {:.2f} sec".format(write_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

write_duration: 44.36 sec

In [19]:
#  set the table location in Spark config, which allows you to read the data using SQL format
spark.conf.set('table.location', config['target'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
# Read the data from Loaction
start_time=time.time()
df_delta = spark.read.format("delta").load(config['target'])
end_time=time.time()
read_duration=end_time-start_time
print("write_duration: {:.2f} sec".format(read_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

write_duration: 0.19 sec

In [22]:
# create DeltaTable instances using the path of the Delta table
deltaTable = DeltaTable.forPath(spark, config['target'])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
# Changing data for upsert
upsert_dest_delta= ["noida" for i in range (0,100)]
df3_delta=create_json_df(spark, get_json_data(1000000,100, upsert_dest_delta))
df3_delta.show(20,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-------+-----------------------+
|destination|route_id|trip_id|ts                     |
+-----------+--------+-------+-----------------------+
|noida      |A       |1000000|2023-05-18 11:29:03.871|
|noida      |B       |1000001|2023-05-18 11:29:03.871|
|noida      |C       |1000002|2023-05-18 11:29:03.871|
|noida      |D       |1000003|2023-05-18 11:29:03.871|
|noida      |E       |1000004|2023-05-18 11:29:03.871|
|noida      |F       |1000005|2023-05-18 11:29:03.871|
|noida      |G       |1000006|2023-05-18 11:29:03.871|
|noida      |H       |1000007|2023-05-18 11:29:03.871|
|noida      |I       |1000008|2023-05-18 11:29:03.871|
|noida      |J       |1000009|2023-05-18 11:29:03.871|
|noida      |A       |1000010|2023-05-18 11:29:03.871|
|noida      |B       |1000011|2023-05-18 11:29:03.871|
|noida      |C       |1000012|2023-05-18 11:29:03.871|
|noida      |D       |1000013|2023-05-18 11:29:03.871|
|noida      |E       |1000014|2023-05-18 11:29:03.871|
|noida    

In [24]:
# PERFORM THE UPSERT OPERATION
start_time=time.time()
(deltaTable
.alias('t')
.merge(df3_delta.alias('u'), 't.trip_id = u.trip_id')
.whenMatchedUpdateAll()
.whenNotMatchedInsertAll()
.execute())
end_time=time.time()
upsert_duration=end_time-start_time
print("write_duration: {:.2f} sec".format(upsert_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

write_duration: 37.39 sec

In [25]:
# df_delta.filter(col('trip_id').between(1000000,1000200)).show(200,False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
# Inserting new data
insert_dest=["ghaziabad" for i in range(0,100)]
df5_mor=create_json_df(spark,get_json_data(2000000,100,insert_dest))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# PERFORM THE INSERT OPERATION
start_time=time.time()
(deltaTable
.alias('t')
.merge(df5_mor.alias('u'), 't.trip_id = u.trip_id')
.whenMatchedUpdateAll()
.whenNotMatchedInsertAll()
.execute())
end_time=time.time()
insert_duration=end_time-start_time
print("insert_duration: {:.2f} sec".format(insert_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

insert_duration: 14.96 sec

In [30]:
df_delta.filter(col('trip_id')>2000000).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+--------+-------+--------------------+
|destination|route_id|trip_id|                  ts|
+-----------+--------+-------+--------------------+
|  ghaziabad|       I|2000048|2023-05-18 11:32:...|
|  ghaziabad|       B|2000031|2023-05-18 11:32:...|
|  ghaziabad|       H|2000047|2023-05-18 11:32:...|
|  ghaziabad|       G|2000006|2023-05-18 11:32:...|
|  ghaziabad|       J|2000019|2023-05-18 11:32:...|
|  ghaziabad|       H|2000027|2023-05-18 11:32:...|
|  ghaziabad|       G|2000016|2023-05-18 11:32:...|
|  ghaziabad|       B|2000041|2023-05-18 11:32:...|
|  ghaziabad|       H|2000037|2023-05-18 11:32:...|
|  ghaziabad|       I|2000008|2023-05-18 11:32:...|
|  ghaziabad|       D|2000013|2023-05-18 11:32:...|
|  ghaziabad|       J|2000009|2023-05-18 11:32:...|
|  ghaziabad|       E|2000004|2023-05-18 11:32:...|
|  ghaziabad|       D|2000003|2023-05-18 11:32:...|
|  ghaziabad|       A|2000030|2023-05-18 11:32:...|
|  ghaziabad|       C|2000002|2023-05-18 11:32:...|
|  ghaziabad

In [37]:
# Time travel 
deltaTable.history(100).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|version|          timestamp|userId|userName|operation| operationParameters| job|notebook|clusterId|readVersion|isolationLevel|isBlindAppend|    operationMetrics|userMetadata|          engineInfo|
+-------+-------------------+------+--------+---------+--------------------+----+--------+---------+-----------+--------------+-------------+--------------------+------------+--------------------+
|      2|2023-05-18 11:32:33|  null|    null|    MERGE|{predicate -> (t....|null|    null|     null|          1|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|      1|2023-05-18 11:29:55|  null|    null|    MERGE|{predicate -> (t....|null|    null|     null|          0|  Serializable|        false|{numTargetRowsCop...|        null|Apache-Spark/3.3....|
|      0|2023-0

In [38]:
deltaTable.history(100).select("version", "timestamp", "operation", "operationParameters").show(truncate=False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+
|version|timestamp          |operation|operationParameters                                                                                                                      |
+-------+-------------------+---------+-----------------------------------------------------------------------------------------------------------------------------------------+
|2      |2023-05-18 11:32:33|MERGE    |{predicate -> (t.trip_id = u.trip_id), matchedPredicates -> [{"actionType":"update"}], notMatchedPredicates -> [{"actionType":"insert"}]}|
|1      |2023-05-18 11:29:55|MERGE    |{predicate -> (t.trip_id = u.trip_id), matchedPredicates -> [{"actionType":"update"}], notMatchedPredicates -> [{"actionType":"insert"}]}|
|0      |2023-05-18 11:27:31|WRITE    |{mode -> Overwrite, partitionBy -> []}                                 

In [39]:
df_time_travel = spark.read.format("delta").option("versionAsOf", 1).load(config['target'])
df_time_travel.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+--------+-------+--------------------+
|  destination|route_id|trip_id|                  ts|
+-------------+--------+-------+--------------------+
|        Miami|       I|     28|2023-05-18 11:23:...|
| Philadelphia|       H|    107|2023-05-18 11:23:...|
|San Francisco|       J|    139|2023-05-18 11:23:...|
| Philadelphia|       H|    157|2023-05-18 11:23:...|
|    Las Vegas|       E|    174|2023-05-18 11:23:...|
|        Miami|       I|    178|2023-05-18 11:23:...|
|        Miami|       I|    208|2023-05-18 11:23:...|
| Philadelphia|       H|    227|2023-05-18 11:23:...|
|   New Jersey|       C|    242|2023-05-18 11:23:...|
|  Los Angeles|       D|    253|2023-05-18 11:23:...|
|       Tucson|       F|    255|2023-05-18 11:23:...|
|Washington DC|       G|    266|2023-05-18 11:23:...|
|        Miami|       I|    268|2023-05-18 11:23:...|
| Philadelphia|       H|    307|2023-05-18 11:23:...|
|Washington DC|       G|    316|2023-05-18 11:23:...|
|      Seattle|       A|    

In [40]:
#NOT WORKING - version  error
#DLETE OPERATION
# start_time=time.time()
# condition=[i for i in range(2000050,2000101)]
# deltaTable.delete(col('trip_id').isin(condition))
# end_time=time.time()
# delete_duration=end_time-start_time
# print("delete_duration: {:.2f} sec".format(delete_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
# DELETE OPERATION
start_time=time.time()
condition="trip_id > 2000050"
deltaTable.delete(condition)
end_time=time.time()
delete_duration=end_time-start_time
print("delete_duration: {:.2f} sec".format(delete_duration))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

An error was encountered:
An error occurred while calling o126.delete.
: java.lang.NoSuchMethodError: org.apache.spark.sql.catalyst.plans.logical.DeleteFromTable.<init>(Lorg/apache/spark/sql/catalyst/plans/logical/LogicalPlan;Lscala/Option;)V
	at io.delta.tables.execution.DeltaTableOperations.$anonfun$executeDelete$1(DeltaTableOperations.scala:42)
	at org.apache.spark.sql.delta.util.AnalysisHelper.improveUnsupportedOpError(AnalysisHelper.scala:105)
	at org.apache.spark.sql.delta.util.AnalysisHelper.improveUnsupportedOpError$(AnalysisHelper.scala:91)
	at io.delta.tables.DeltaTable.improveUnsupportedOpError(DeltaTable.scala:44)
	at io.delta.tables.execution.DeltaTableOperations.executeDelete(DeltaTableOperations.scala:41)
	at io.delta.tables.execution.DeltaTableOperations.executeDelete$(DeltaTableOperations.scala:41)
	at io.delta.tables.DeltaTable.executeDelete(DeltaTable.scala:44)
	at io.delta.tables.DeltaTable.delete(DeltaTable.scala:173)
	at sun.reflect.NativeMethodAccessorImpl.invoke

In [31]:
condition=[i for i in range(2000050,2000100)]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
condition


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2000050, 2000051, 2000052, 2000053, 2000054, 2000055, 2000056, 2000057, 2000058, 2000059, 2000060, 2000061, 2000062, 2000063, 2000064, 2000065, 2000066, 2000067, 2000068, 2000069, 2000070, 2000071, 2000072, 2000073, 2000074, 2000075, 2000076, 2000077, 2000078, 2000079, 2000080, 2000081, 2000082, 2000083, 2000084, 2000085, 2000086, 2000087, 2000088, 2000089, 2000090, 2000091, 2000092, 2000093, 2000094, 2000095, 2000096, 2000097, 2000098, 2000099]